# Multivariate forecasting
> Tutorial on how to do multivariate forecasting using TSMixer models.

In _multivariate_ forecasting, we use the information from every time series to produce all forecasts for all time series jointly. In contrast, in _univariate_ forecasting we only consider the information from every individual time series and produce forecasts for every time series separately. Multivariate forecasting methods thus use more information to produce every forecast, and thus should be able to provide better forecasting results. However, multivariate forecasting methods also scale with the number of time series, which means these methods are commonly less well suited for large-scale problems (i.e. forecasting many, many time series).

In this notebook, we will demonstrate the performance of a state-of-the-art multivariate forecasting architecture `TSMixer` / `TSMixerx` when compared to a univariate forecasting method (`NHITS`).

We will show how to:
* Load the [ETTm2](https://github.com/zhouhaoyi/ETDataset) benchmark dataset, used in the academic literature.
* Train a TSMixer and TSMixerx model
* Forecast the test set
* Optimize the hyperparameters

You can run these experiments using GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/neuralforecast/blob/main/nbs/examples/LongHorizon_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installing libraries

In [ ]:
%%capture
!pip install neuralforecast datasetsforecast

## 2. Load ETTm2 Data

The `LongHorizon` class will automatically download the complete ETTm2 dataset and process it.

It return three Dataframes: `Y_df` contains the values for the target variables, `X_df` contains exogenous calendar features and `S_df` contains static features for each time-series (none for ETTm2). For this example we will use `Y_df` and `X_df`. 

In `TSMixerx`, we can make use of the additional exogenous features contained in `X_df`. In `TSMixer`, there is _no_ support for exogenous features. Hence, if you want to use exogenous features, you should use `TSMixerx`. 

If you want to use your own data just replace `Y_df` and `X_df`. Be sure to use a long format and make sure to have a similar structure as our data set.

In [ ]:
import pandas as pd

from datasetsforecast.long_horizon import LongHorizon

In [ ]:
# Change this to your own data to try the model
Y_df, X_df, _ = LongHorizon.load(directory='./', group='ETTm2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

# X_df contains the exogenous features, which we add to Y_df
X_df['ds'] = pd.to_datetime(X_df['ds'])
Y_df = Y_df.merge(X_df, on=['unique_id', 'ds'], how='left')

# We make validation and test splits
n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.2 * n_time)

In [ ]:
Y_df

## 3. Train models

We will train models using the `cross_validation` method, which allows users to automatically simulate multiple historic forecasts (in the test set).

The `cross_validation` method will use the validation set for hyperparameter selection and early stopping, and will then produce the forecasts for the test set.

First, instantiate each model in the `models` list, specifying the `horizon`, `input_size`, and training iterations. In this notebook, we compare against the `NHITS` model.

In [ ]:
# %%capture
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TSMixer, TSMixerx, NHITS
from neuralforecast.losses.pytorch import MSE, MAE

In [ ]:
%%capture
horizon = 96
input_size = 512
models = [
          TSMixer(h=horizon,
                input_size=input_size,
                n_series=7,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=5,
                scaler_type='identity',
                valid_loss=MAE(),
                random_seed=12345678,
                ),  
          TSMixerx(h=horizon,
                input_size=input_size,
                n_series=7,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=5,
                scaler_type='identity',
                dropout=0.7,
                valid_loss=MAE(),
                random_seed=12345678,
                futr_exog_list=['ex_1', 'ex_2', 'ex_3', 'ex_4'],
                ),                          
           NHITS(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=5,
                scaler_type='robust',
                valid_loss=MAE(),
                random_seed=12345678,
                ),                                                                       
         ]

:::{.callout-tip}
Check our `auto` models for automatic hyperparameter optimization, and see the end of this tutorial for an example of hyperparameter tuning.
:::

Instantiate a `NeuralForecast` object with the following required parameters:

* `models`: a list of models.

* `freq`: a string indicating the frequency of the data. (See [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

Second, use the `cross_validation` method, specifying the dataset (`Y_df`), validation size and test size.

In [ ]:
%%capture
nf = NeuralForecast(
    models=models,
    freq='15min')

Y_hat_df = nf.cross_validation(df=Y_df,
                               val_size=val_size,
                               test_size=test_size,
                               n_windows=None
                               )                                 
Y_hat_df = Y_hat_df.reset_index()                               

The `cross_validation` method will return the forecasts for each model on the test set.

## 4. Evaluate Results

Next, we plot the forecasts on the test set for the `OT` variable for all models.

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')
plt.plot(Y_plot['ds'], Y_plot['TSMixer'], label='TSMixer')
plt.plot(Y_plot['ds'], Y_plot['TSMixerx'], label='TSMixerx')
plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')
plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

Finally, we compute the test errors using the Mean Absolute Error (MAE) and Mean Squared Error (MSE):

$\qquad MAE = \frac{1}{Windows * Horizon} \sum_{\tau} |y_{\tau} - \hat{y}_{\tau}| \qquad$ and $\qquad MSE = \frac{1}{Windows * Horizon} \sum_{\tau} (y_{\tau} - \hat{y}_{\tau})^{2} \qquad$

In [ ]:
from neuralforecast.losses.numpy import mse, mae

mae_tsmixer = mae(Y_hat_df['y'], Y_hat_df['TSMixer'])
mse_tsmixer = mse(Y_hat_df['y'], Y_hat_df['TSMixer'])
mae_tsmixerx = mae(Y_hat_df['y'], Y_hat_df['TSMixerx'])
mse_tsmixerx = mse(Y_hat_df['y'], Y_hat_df['TSMixerx'])
mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])


print(f'TSMixer horizon {horizon} - MAE: {mae_tsmixer:.3f}')
print(f'TSMixer horizon {horizon} - MSE: {mse_tsmixer:.3f}')
print(f'TSMixerx horizon {horizon} - MAE: {mae_tsmixerx:.3f}')
print(f'TSMixerx horizon {horizon} - MSE: {mse_tsmixerx:.3f}')
print(f'NHITS horizon {horizon} - MAE: {mae_nhits:.3f}')
print(f'NHITS horizon {horizon} - MSE: {mse_nhits:.3f}')

For reference, we can check the performance when compared to self-reported performance in the paper. We find that `TSMixer` provides better results than the _univariate_ method `NHITS`. Also, our implementation of `TSMixer` very closely tracks the results of the original paper. Finally, it seems that there is little benefit of using the additional exogenous variables contained in the dataframe `X_df` as `TSMixerx` performs worse than `TSMixer`, especially on longer horizons. 

Mean Absolute Error (MAE)

| Horizon   |TSMixer<br> (this notebook) | TSMixer <br>(paper) | TSMixerx<br> (this notebook) |  NHITS <br>(this notebook)    | NHITS <br>(paper) 
|---        |---        |---        |---        |---        |---       
|  96       | **0.250** | 0.252     | 0.257     |  0.251    |   0.255      
|  192      | **0.288** | 0.290     | 0.300     |  0.291    |   0.305      
|  336      | **0.323** | 0.324     | 0.380     |  0.344    |   0.346      
|  720      | **0.377** | 0.422     | 0.464     |  0.417    |   0.413     

Mean Squared Error (MSE)

| Horizon   |TSMixer<br> (this notebook) | TSMixer <br>(paper) | TSMixerx<br> (this notebook) | NHITS <br>(this notebook)    | NHITS <br>(paper) 
|---        |---        |---            |---        |---                |---            
|  96       | **0.163** | **0.163**     | 0.170     |  0.179            |   0.176      
|  192      | 0.220     | **0.216**     | 0.231     |  0.239            |   0.245       
|  336      | 0.272     | **0.268**     | 0.361     |  0.311            |   0.295       
|  720      | **0.356** | 0.420         | 0.493     |  0.451            |   0.401      

Note that for the table above, we use the same hyperparameters for all methods for all horizons, whereas the original papers tune the hyperparameters for each horizon.

## 5. Tuning the hyperparameters
The `AutoTSMixer` / `AutoTSMixerx` class will automatically perform hyperparamter tunning using the [Tune library](https://docs.ray.io/en/latest/tune/index.html), exploring a user-defined or default search space. Models are selected based on the error on a validation set and the best model is then stored and used during inference. 

The `AutoTSMixer.default_config` / `AutoTSMixerx.default_config`  attribute contains a suggested hyperparameter space. Here, we specify a different search space following the paper's hyperparameters. Feel free to play around with this space.

For this example, we will optimize the hyperparameters for `horizon = 96`.

In [ ]:
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from neuralforecast.auto import AutoTSMixer, AutoTSMixerx

horizon = 96 # 24hrs = 4 * 15 min.

tsmixer_config = {
       "input_size": input_size,                                                 # Size of input window
       "max_steps": tune.choice([500, 1000, 2000]),                              # Number of training iterations
       "val_check_steps": 100,                                                   # Compute validation every x steps
       "early_stop_patience_steps": 5,                                           # Early stopping steps
       "learning_rate": tune.loguniform(1e-4, 1e-2),                             # Initial Learning rate
       "n_block": tune.choice([1, 2, 4, 6, 8]),                                  # Number of mixing layers
       "dropout": tune.uniform(0.0, 0.99),                                       # Dropout
       "ff_dim": tune.choice([32, 64, 128]),                                     # Dimension of the feature linear layer
       "scaler_type": 'identity',       
    }

tsmixerx_config = tsmixer_config.copy()
tsmixerx_config['futr_exog_list'] = ['ex_1', 'ex_2', 'ex_3', 'ex_4']

To instantiate `AutoTSMixer` and `AutoTSMixerx` you need to define:

* `h`: forecasting horizon
* `n_series`: number of time series in the multivariate time series problem.

In addition, we define the following parameters (if these are not given, the `AutoTSMixer`/`AutoTSMixerx` class will use a pre-defined value):
* `loss`: training loss. Use the `DistributionLoss` to produce probabilistic forecasts.
* `config`: hyperparameter search space. If `None`, the `AutoTSMixer` class will use a pre-defined suggested hyperparameter space.
* `num_samples`: number of configurations explored. For this example, we only use a limited amount of `10`.
* `search_alg`: type of search algorithm used for selecting parameter values within the hyperparameter space.
* `backend`: the backend used for the hyperparameter optimization search, either `ray` or `optuna`. 
* `valid_loss`: the loss used for the validation sets in the optimization procedure.

In [ ]:
model = AutoTSMixer(h=horizon,
                    n_series=7,
                    loss=MAE(),
                    config=tsmixer_config,
                    num_samples=10,
                    search_alg=HyperOptSearch(),
                    backend='ray',
                    valid_loss=MAE())

modelx = AutoTSMixerx(h=horizon,
                    n_series=7,
                    loss=MAE(),
                    config=tsmixerx_config,
                    num_samples=10,
                    search_alg=HyperOptSearch(),
                    backend='ray',
                    valid_loss=MAE())

Now, we fit the model by instantiating a `NeuralForecast` object with the following required parameters:

* `models`: a list of models.

* `freq`: a string indicating the frequency of the data. (See [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

The `cross_validation` method allows you to simulate multiple historic forecasts, greatly simplifying pipelines by replacing for loops with `fit` and `predict` methods.

With time series data, cross validation is done by defining a sliding window across the historical data and predicting the period following it. This form of cross validation allows us to arrive at a better estimation of our model’s predictive abilities across a wider range of temporal instances while also keeping the data in the training set contiguous as is required by our models.

The `cross_validation` method will use the validation set for hyperparameter selection, and will then produce the forecasts for the test set.

In [ ]:
%%capture
nf = NeuralForecast(models=[model, modelx], freq='15min')
Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size,
                               test_size=test_size, n_windows=None)

## 6. Evaluate Results

The `AutoTSMixer`/`AutoTSMixerx` class contains a `results` attribute that stores information of each configuration explored. It contains the validation loss and best validation hyperparameter. The result dataframe `Y_hat_df` that we obtained in the previous step is based on the best config of the hyperparameter search. For `AutoTSMixer`, the best config is:

In [ ]:
nf.models[0].results.get_best_result().config

and for `AutoTSMixerx`:

In [ ]:
nf.models[1].results.get_best_result().config

We compute the test errors of the best config for the two metrics of interest:

$\qquad MAE = \frac{1}{Windows * Horizon} \sum_{\tau} |y_{\tau} - \hat{y}_{\tau}| \qquad$ and $\qquad MSE = \frac{1}{Windows * Horizon} \sum_{\tau} (y_{\tau} - \hat{y}_{\tau})^{2} \qquad$

In [ ]:
y_true = Y_hat_df.y.values
y_hat_tsmixer = Y_hat_df['AutoTSMixer'].values
y_hat_tsmixerx = Y_hat_df['AutoTSMixerx'].values

print(f'MAE TSMixer: {mae(y_hat_tsmixer, y_true):.3f}')
print(f'MSE TSMixer: {mse(y_hat_tsmixer, y_true):.3f}')
print(f'MAE TSMixerx: {mae(y_hat_tsmixerx, y_true):.3f}')
print(f'MSE TSMixerx: {mse(y_hat_tsmixerx, y_true):.3f}')

We can compare the error metrics for our optimized setting to the earlier setting in which we used the default hyperparameters. In this case, for a horizon of 96, we got slightly improved results for `TSMixer` on `MAE`. Interestingly, we did not improve for `TSMixerx` as compared to the default settings. For this dataset, it seems there is limited value in using exogenous features with the `TSMixerx` architecture for a horizon of 96.

| Metric    |TSMixer<br> (optimized) | TSMixer <br>(default)  | TSMixer <br>(paper)   |TSMixerx<br> (optimized) | TSMixerx <br>(default) 
|---        |---                     |---                     |---                    |---                      |---
| MAE       | **0.247**              | 0.250                  | 0.252                 | 0.258                   | 0.257 
| MSE       | **0.162**              | 0.163                  | **0.163**             | 0.174                   | 0.170

Note that we only evaluated 10 hyperparameter configurations (`num_samples=10`), which may suggest that it is possible to further improve forecasting performance by exploring more hyperparameter configurations.

## References

[Chen, Si-An, Chun-Liang Li, Nate Yoder, Sercan O. Arik, and Tomas Pfister (2023). "TSMixer: An All-MLP Architecture for Time Series Forecasting."](http://arxiv.org/abs/2303.06053) <br>
[Cristian Challu, Kin G. Olivares, Boris N. Oreshkin, Federico Garza, Max Mergenthaler-Canseco, Artur Dubrawski (2021). NHITS: Neural Hierarchical Interpolation for Time Series Forecasting. Accepted at AAAI 2023.](https://arxiv.org/abs/2201.12886)